In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt = load_prompt("./prompt.json")
prompt_yaml = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

prompt.format(country="Germany")
prompt_yaml.format(country="Germany")

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [("intro", intro), ("example", example), ("start", start)]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt.format(
#     charactor="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrg!!!! That is a secret! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

chain.invoke(
    {
        "charactor": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrg!!!! That is a secret! Arg arg!!",
        "question": "What is your fav food?",
    }
)